In [1]:
# Used to access the saved Chroma vector database. 
from langchain_community.vectorstores import Chroma

# Used to embed the user's query with text-embedding-3-small 
# and send it to gpt-4o-mini.
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Used to get the statistics of each prompt, notably the input tokens,
# output tokens, and associated cost.
from langchain_community.callbacks import get_openai_callback

# Used to get the OpenAI API key from environment variables so that 
# it's not visible in this code.
import os

In [2]:
# Get the API key.
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

# I chose not to use Pinecone, as it adds considerable complication to the code.
# os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY")

# Sets the directory of the Chroma DB that's being loaded from.
# At present, it's just "Chroma", but I may make seperate DBs for different embedding models.
CHROMA_PATH = "Chroma"

In [3]:
# The first step of engineering the prompt. The LLM is told to answer the question with 
# the given context. Currently, there isn't any. However, this will be formatted later
# to take this string and add the context of the relevant database chunk in place of {context}
# and add the user's query in place of {question}.
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [4]:
# Sets up the embedding model with the API key.
# Conveniently, LangChain allows for "hot-swapping" of embedding models by merely
# changing the model argument. However, using a different embedding model than the one 
# used to create the vector database will have significant negative consequences that could 
# render the chatbot inoperable, so it's essential that this matches what's used in Chroma.py.
embedder = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = os.environ["OPENAI_API_KEY"]
)

In [5]:
# Load the vector database.
db = Chroma(persist_directory = CHROMA_PATH, embedding_function = embedder)

C:\Users\Lewis\AppData\Local\Temp\ipykernel_34256\962685837.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory = CHROMA_PATH, embedding_function = embedder)


In [9]:
# CURRENTLY, THIS IS THE ONLY WAY OF QUERYING THE DB AND LLM.
# I'm looking into a package called "Streamlit", which provides a web interface for user input.
# Alternatively, I think it'd be good to somehow get this as a bot on a messaging service (Teams, Discord)
# and allow queries to flow in from there. (I have yet to do any research on Teams or Discord LLM bots.)
query = "What is the university's approach to teaching?"

In [ ]:
# Finds the three most likely chunks that the user's query applies to.
results = db.similarity_search_with_relevance_scores(query, k=3)

for doc, _score in results:
    #print(doc)
    print(_score)

# Creates the {context} previously seen in the PROMPT_TEMPLATE.
# Seperates the three most likely chunks with new lines and dashes.
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

# Formats the prompt to put the document context in place of {context},
# and the user query in place of {question}
prompt = PROMPT_TEMPLATE.format(context=context_text, question=query)
print(prompt)

0.23076361850953075
0.2226055118480279
0.20876995397096276

Answer the question based only on the following context:

5.

Principles

1. The University expects students to attend all scheduled learning sessions (on campus and/or online) and to engage with all learning activities, resources and assessments during their studies. Students should be proactive in seeking support if they face any challenges preventing them from engaging with their studies to the expected level.

2. This policy is underpinned by the Student Attendance and Engagement Operational Guidance. The University community and our students all have responsibilities in relation to attendance and engagement as detailed in this guidance, section 8.

3. Attendance at some scheduled learning sessions (e.g., laboratory sessions, studio sessions, workshops) may be mandatory due to PSRB requirements. The student Course Handbook will make clear where there are any enhanced attendance requirements and any consequences of non-atte

In [ ]:
# Used to format the LLM's output response. 
# Completely unnecessary, just makes this notebook look nicer.
from IPython.display import display, Markdown

# Create the LLM object.
## Uses GPT-4o-mini for cost efficiency.
## Temperature can be changed to vary the LLMs responses.
## Loads the API key from the system environment variables.
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0,
    api_key = os.environ["OPENAI_API_KEY"]
)

# To track the stats (input/output tokens, cost) of each prompt,
# get the API callback.
with get_openai_callback() as cb:
    # Gets the LLM's response.
    response = llm.invoke(prompt)

    # Attaches the document that the relevant chunks were found from.
    sources = [doc.metadata.get("source", None) for doc, _score in results]

    # Saves and outputs the LLM's response, and the sources used to generate it.
    formatted_response = f"""{llm.model_name} says:  
        {response.content}  
        Sources: {sources}"""
    display(Markdown(formatted_response))

    # Code from LangChain example (https://python.langchain.com/docs/how_to/llm_token_usage_tracking/)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion/Output Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    
### CURRENTLY, THIS IS QUERYING A DB GENERATED FROM POLICIES/TESTING, AS CHROMA DOESN'T WORK WITH MANY DOCUMENTS?
### Likely alleviated with PyPDFLoader as used before.

gpt-4o-mini says:  
        The university's approach to teaching emphasizes the importance of attendance and engagement in scheduled learning sessions, whether on campus or online. It expects students to actively participate in all learning activities, resources, and assessments, and encourages them to seek support if they encounter challenges. The university monitors attendance from the first week of teaching and implements both informal and formal interventions if attendance or engagement is a concern. Additionally, certain sessions may have mandatory attendance requirements due to professional, statutory, and regulatory body (PSRB) obligations. Overall, the university aims to support students in maximizing their potential through proactive engagement and communication.  
        Sources: ['Data\\Policies\\TESTING\\Attendance.pdf', 'Data\\Policies\\TESTING\\Attendance.pdf', 'Data\\Policies\\TESTING\\Attendance.pdf']

Total Tokens: 538
Prompt Tokens: 419
Completion/Output Tokens: 119
Total Cost (USD): $0.00013424999999999998
